In [2]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import platform
from bs4 import BeautifulSoup
from matplotlib import font_manager, rc


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Plot 한글 지원
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Linux':
    path = "/usr/share/fonts/NanumGothic.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
else:
    print('Unknown system... ')

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = "https://www.laphil.com/events/performances"
driver.get(url)

# for scrolling
current_scroll_position = driver.execute_script("return window.pageYOffset;")
scroll_height = driver.execute_script("return document.body.scrollHeight;")
if current_scroll_position < scroll_height:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
driver.implicitly_wait(10)

soup = BeautifulSoup( driver.page_source , 'html.parser')
titles = soup.find_all(class_="info")

# Dislay raw lines
print(titles)
for title in titles:
    print(title.text.strip())



[<div class="info">
<p class="name-container">
<a class="name name--short" data-auto-translate="true" href="https://www.laphil.com/events/performances/2468/2024-03-07/prokofiev-and-rachmaninoff">
                            Prokofiev and Rachmaninoff
                            
                        </a>
</p>
<div class="date" data-auto-translate="true">
<div class="date-text">Thu, Mar 7</div> <div class="attendance">You are going!</div>
</div>
<p class="time" data-auto-translate="true">
                            8:00PM
                            
                        </p>
<div class="toggle-more" data-auto-translate="true" data-js-ui="toggle-performance-info">Program &amp; Artist Listing</div>
<div class="performance-card__program-and-artists">
<div class="performance-card__program-and-artists-info">
<div class="performance-card__program-and-artists-section" data-auto-translate="true">
<p>
<strong>
<span>Sofia GUBAIDULINA</span>
</strong> —
                                   

In [6]:
data = []
titles = soup.find_all(class_="info")
for title in titles:
    # bring title text
    title_text = title.find(class_="name name--short").get_text(strip=True)
    # bring supporting-acts text
    supporting_acts = title.find(class_="supporting-acts")
    supporting_acts_text = (
        supporting_acts.get_text(strip=True) if supporting_acts else ""
    )
    # parse text
    date_text = title.find(class_="date-text").get_text(strip=True)
    time_text = title.find(class_="time").get_text(strip=True).replace("\n", " ")
    time_text = " ".join(time_text.split())

    # append to Pandas DataFrame
    data.append({"title": title_text, "date": date_text, "time": time_text})
    # print(title_text + " " + supporting_acts_text + " " + date_text + " " + time_text)

df = pd.DataFrame(data)
print(df)



                                                 title         date     time
0                           Prokofiev and Rachmaninoff   Thu, Mar 7   8:00PM
1            Swan Lake: ReimaginedSymphonies for Youth   Sat, Mar 9  11:00AM
2                           Prokofiev and Rachmaninoff   Sat, Mar 9   8:00PM
3                           Prokofiev and Rachmaninoff  Sun, Mar 10   2:00PM
4            Supermanin ConcertJohn Williams Spotlight  Fri, Mar 15   8:00PM
..                                                 ...          ...      ...
437                              Tchaikovsky & Pereira  Sun, June 1   2:00PM
438  New Voices from KoreaSeoul FestivalGreen Umbre...  Tue, June 3   8:00PM
439  Seoul Festival with the LA Phil: Night 1Seoul ...  Fri, June 6   8:00PM
440  Seoul Festival with the LA Phil: Night 2Seoul ...  Sat, June 7   8:00PM
441  Seoul Festival with the LA Phil: Night 2Seoul ...  Sun, June 8   2:00PM

[442 rows x 3 columns]
